In [49]:
%%HTML
<style>
    h1 {color: yellow;}
    table td {border:2px solid white !important;}
</style>

# Bitboard Engines

Bitboards are an alternative representation of the state of a checker board. A bitboard is a 32 bit word used to represent the 32 legal squares. Each bit is used to represent one element of each square. Each checker board position can be represented with 3 bitboards: black pieces, red pieces, and kings. A bitboard of empty squares can then be derived from ~(red pieces | black pieces).

Some resources:
- [Jonathan Kreuzer](https://www.3dkingdoms.com/checkers/bitboards.htm) wrote a tutorial that helped get started.
- But see [this](https://www.talkchess.com/forum3/viewtopic.php?t=64487) suggesting that Kreuzer's bitboard was not the best way. That using 43 bit ints would make all pieces move by 4 and 5. Rather than some by 3 and 4 and others by 4 and 5, depending on the rank of the square.
- [Counting set bits](https://www.geeksforgeeks.org/count-set-bits-in-an-integer/)
- [Get the position of the first set bit](https://btechgeeks.com/python-program-to-find-position-of-rightmost-set-bit/)
- [Modify a bit at a given position](https://www.geeksforgeeks.org/modify-bit-given-position/)

|littleBitA| board2 padded array |
| ------- | ------------------- |
|&emsp;00&emsp;01&emsp;02&emsp;03 | &emsp;37&emsp;38&emsp;39&emsp;40 |
|04&emsp;05&emsp;06&emsp;07       | 32&emsp;33&emsp;34&emsp;35       |
|&emsp;08&emsp;09&emsp;10&emsp;11 | &emsp;28&emsp;29&emsp;30&emsp;31 |
|12&emsp;13&emsp;14&emsp;15       | 23&emsp;24&emsp;25&emsp;26       | 
|&emsp;16&emsp;17&emsp;18&emsp;19 | &emsp;19&emsp;20&emsp;21&emsp;22 |
|20&emsp;21&emsp;22&emsp;23       | 14&emsp;15&emsp;16&emsp;17       |
|&emsp;24&emsp;25&emsp;26&emsp;27 | &emsp;10&emsp;11&emsp;12&emsp;13 |
|28&emsp;29&emsp;30&emsp;31       | 05&emsp;06&emsp;07&emsp;08       |



## littleBitA Engine

littleBitA uses python integers rather than numpy entities. This about 3x faster. Somehow, littleBit was not using the numpy integers efficiently. 

One idea is to try to numpy rshift and lshift methods instead of the native python methods. I temporarily modified litteBit to use the numpy methods, but there was no noticeable speed improvement.

In [29]:
import timeit
import engines
from board2 import Board

i = 100000
b = Board()

lba = engines.littlebitA(b)
position = lba.convPos2BB()
movers = lba.getMovers(position)

lb = engines.littlebit(b)
lb.convert2BB(b.pos2Fen())
lbmovers = lb.getMovers()

time4PadArray = timeit.timeit(b.getLegalMoves, number = i)
time4Bitboard = timeit.timeit(lambda: lba.getNormalMoves(position, movers), number = i)
time4lba = timeit.timeit(lambda: lb.getNormalMoves(lbamovers), number = i)

print(f'Padded Array Time:\t{time4PadArray:.2f}')
print(f'LittleBitA Time:\t{time4Bitboard:.2f}')
print(f'LittleBit Time:\t\t{time4lba:.2f}')
print(f'LittleBitA is {time4PadArray/time4Bitboard:.2f} times faster than the padded array method.')

print(lba.getNormalMoves(position, movers))

Padded Array Time:	2.37
LittleBitA Time:	0.75
LittleBit Time:		3.10
LittleBitA is 3.17 times faster than the padded array method.
[(8, 12), (8, 13), (9, 13), (9, 14), (10, 14), (10, 15), (11, 15)]


## littleBit Engine

In this first effort at using a bitboard representation, I somehow got the idea to use numpy unsigned 32 bit objects (numpy.uint32) instead of the standard python int. Suprisingly, this was a little slower than board2.Board at finding non-jump moves even though:
- board2.Board has to iterate over all 32 squares looking for the men on move
- littleBit is concerned only with the 4 squares where it knows there are movers

In [9]:
import timeit
import engines
from board2 import Board

i = 100000
b = Board()
e = engines.littlebit(b)
e.convert2BB(b.pos2Fen())
movers = e.getMovers()

time4PadArray = timeit.timeit(b.getLegalMoves, number = i)
time4Bitboard = timeit.timeit(lambda: e.getNormalMoves(movers), number = i)

print(f'Padded Array:\t{time4PadArray} \nBitboard:\t{time4Bitboard}')

Padded Array:	2.6310233999975026 
Bitboard:	3.2120468000066467
